In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_path = 'C:/Users/ticom/VLabs/Data'
df = pd.read_csv(data_path+'/sales_20_21_train.csv', header=0, parse_dates=['DT_VENDA'])
df_sub = pd.read_csv(data_path+'/sample_submission.csv',header=0)

# Prophet

In [ ]:
model= Prophet(**model_params)
data['cap']=data['y'].max() + data['y'].std()*0.05

In [ ]:
model_params = {'daily_seasonality':False,
               'weekly_seasonality':False,
               'yearly_seasonality': True,
               'seasonality_mode':'multiplicative',
               'growth':'logistic'}

In [32]:
model= Prophet(**model_params)
data['cap']=data['y'].max() + data['y'].std()*0.05

model.fit(data)

SyntaxError: unexpected character after line continuation character (Temp/ipykernel_2296/1581457418.py, line 2)

In [ ]:
future = model.make_future_dataframe(periods=90)
future['cap'] = min(data['cap'].max()*2,150)

forecast = model.predict(future)

In [ ]:
# Prophet

model= Prophet(**model_params)
data['cap']=data['y'].max() + data['y'].std()*0.05

model_params = {'daily_seasonality':False,
               'weekly_seasonality':False,
               'yearly_seasonality': True,
               'seasonality_mode':'multiplicative',
               'growth':'logistic'}

model= Prophet(**model_params)
data['cap']=data['y'].max() + data['y'].std()*0.05

model.fit(data)

future = model.make_future_dataframe(periods=90)
future['cap'] = min(data['cap'].max()*2,150)

forecast = model.predict(future)